## 1. Import Library, Data

In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
!pip install pyspark
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
!apt-get install openjdk-8-jdk-headless -qq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   164  100   164    0     0    842      0 --:--:-- --:--:-- --:--:--   845
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file 
100   146  100   146    0     0    290      0 --:--:-- --:--:-- --:--:--  1417
curl: (23) Failure writing output to destination


In [131]:
import os
import numpy as np
import pandas as pd
import nltk
import pyspark
import pyspark.sql.functions as F
import sys
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from datetime import datetime
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, sum as pyspark_sum

In [132]:
os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64"

# sc= pyspark.SparkContext()
spark= SparkSession.builder.getOrCreate()

In [133]:
path= "/content/drive/MyDrive/23 U 4-2/빅데이터처리및응용/Data/"

movies_org= pd.read_csv(path + "movies.csv")
movies_sample= pd.read_csv(path + "movies_all.csv")
ratings= pd.read_csv(path + "ratings.csv")
tags= pd.read_csv(path + "tags.csv")

In [134]:
na_tags= pd.read_csv(path+ "1210_dfNa_tag.csv")
lzTag= []
for i in range(len(na_tags)):
  if str(na_tags.iloc[i,-1])!='nan':
    tag= na_tags.iloc[i,-3]+' '+ na_tags.iloc[i,-2]+ ' '+ na_tags.iloc[i,-1]
    unique_tag= Counter(tag.split(" "))
    lzTag.append(" ".join(unique_tag.keys()))
  else: lzTag.append(np.nan)
na_tags['tag']= lzTag
na_tags.drop(['tag_1', 'tag_2', 'tag_3'], axis= 1, inplace= True)
na_tags.head()

,movieId,title,tag
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,"shanghai, china chinese mafia coming of age"
1,242,Farinelli: il castrato (1994),sibling relationship opera eunuch
2,243,Gordy (1995),country music pig talking
3,250,Heavyweights (Heavy Weights) (1995),overweight summer camp psychopath
4,251,"Hunted, The (1995)",martial arts assassin self sacrifice


In [135]:
links= pd.read_csv(path+ "links.csv")
meta= pd.read_csv(path+ "movies_metadata.csv", encoding='unicode_escape')
meta.drop(['Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'adult', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'id',
       'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'video',
       'vote_average', 'vote_count',
       'spoken_languages', 'status'], axis=1, inplace= True)
meta.dropna(inplace= True)
for i in range(len(meta)):
  id= meta.iloc[i,0].replace(" ","")
  if len(id)>=3:
    meta.iloc[i,0]= int(id[2:])

## 2. Beforehand Preprocessing


In [136]:
# Remove unnecessary columns
movies_sample.drop(['language', 'director'], axis= 1, inplace= True)

In [137]:
# Merge Tags (tag1,2,3 -> tag)
lzTag= []
for i in range(len(movies_sample)):
  if str(movies_sample.iloc[i,-1])!='nan':
    tag= movies_sample.iloc[i,-3]+' '+ movies_sample.iloc[i,-2]+ ' '+ movies_sample.iloc[i,-1]
    unique_tag= Counter(tag.split(" "))
    lzTag.append(" ".join(unique_tag.keys()))
  else: lzTag.append(np.nan)
movies_sample['tag']= lzTag
movies_sample.drop(['tag_1', 'tag_2', 'tag_3'], axis= 1, inplace= True)
movies_sample.head()

,movieId,title,genres,storyline,country,release_year,runtime,age_rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A little boy named Andy loves to be in his roo...,United States,1995.0,1 hour 21 minutes,All,toy comes to life rivalry
1,2,Jumanji (1995),Adventure|Children|Fantasy,"Jumanji, one of the most unique--and dangerous...",United States,1995.0,1 hour 44 minutes,All,board game trapped in a adventurer
2,3,Grumpier Old Men (1995),Comedy|Romance,Things don't seem to change much in Wabasha Co...,United States,1995.0,1 hour 41 minutes,15,sequel boat lake
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,This story based on the best selling novel by ...,United States,1995.0,2 hours 4 minutes,18,black american friendship between women husban...
4,5,Father of the Bride Part II (1995),Comedy,"In this sequel to ""Father of the Bride"", Georg...",United States,1995.0,1 hour 46 minutes,All,fatherhood doberman dog


In [138]:
# Tag Imputation
tags.drop(['userId'], axis= 1, inplace= True)
lzID= tags['movieId'].unique()
lzTag= []
for id in lzID:
  df= tags[tags['movieId']==id]
  lzTag.append(str(set(df['tag']))[1:-1].replace("'", "").replace(",",""))

tags_uni= pd.DataFrame()
tags_uni['movieId']= lzID
tags_uni['tag']= lzTag
lzExist= list(tags_uni['movieId'].unique())

for i in range(len(movies_sample)):
  mvid= movies_sample.iloc[i,0]
  if str(movies_sample.iloc[i,-1])== 'nan' and mvid in lzExist:
    newTag= tags_uni[tags_uni['movieId']==mvid].iloc[0,1]
    newTag= Counter(newTag.split(" "))
    movies_sample.iloc[i,-1]= " ".join(newTag.keys())

 # Extra Crawling
lzExist= list(na_tags['movieId'].unique())
for i in range(len(movies_sample)):
  id= int(movies_sample.iloc[i,0])
  if (str(movies_sample.iloc[i,-1])=='nan') and (id in lzExist):
    tag= na_tags[na_tags['movieId']==id].iloc[0,-1]
    movies_sample.iloc[i,-1]= tag

 # Use metadata
lzExist= list(meta['imdb_id'].unique())
for i in range(len(movies_sample)):
  id= int(movies_sample.iloc[i,0])
  imdb= links[links['movieId']==id].iloc[0,1]
  if (str(movies_sample.iloc[i,-1])=='nan') and (imdb in lzExist):
    tag= meta[meta['imdb_id']==imdb].iloc[0,1]
    movies_sample.iloc[i,-1]= tag

In [139]:
# Restore TS
time_unit = 3
ratings['timestamp'] = ratings['timestamp'].apply(lambda x: datetime.fromtimestamp(x/time_unit))
tags['timestamp'] = tags['timestamp'].apply(lambda x: datetime.fromtimestamp(x/time_unit))

In [140]:
# Elimination of any blanks in title and tag
for i in range(len(movies_sample)):
  ttl= movies_sample.iloc[i,1].strip()
  tag= movies_sample.iloc[i,-1]

  movies_sample.iloc[i,1]= ttl
  if str(tag)!= 'nan': movies_sample.iloc[i,-1]= tag

In [141]:
movies_sample.isna().sum()

movieId          0
title            0
genres           0
storyline        0
country          0
release_year     0
runtime          0
age_rating       0
tag             45
dtype: int64

In [142]:
movies_sample.dropna(inplace= True)

In [143]:
# Age-rating : rid of reasons
lzAR= []
for i in range(len(movies_sample)):
  ar= movies_sample.iloc[i,-2]
  if any(item in ar for item in ['Unrated', 'Not Rated', 'Approved', 'Passed', 'E']): lzAR.append('All')   # 전체
  elif any(item in ar for item in ['7', 'TV-Y7']): lzAR.append('7')                                    # 7
  elif any(item in ar for item in ['Rated PG-13', '12', 'PG', 'GP', 'Rated PG for', 'PG-13', 'G', '13+', 'TV-13']): lzAR.append('PG-13') # 13
  elif any(item in ar for item in ['TV-14', 'M', 'M/PG', 'NC-17', 'TV-MA', '15', '16+']): lzAR.append('15')  # 15
  elif any(item in ar for item in ['R', 'Rated R', 'Rated NC-17', '18', 'NC-17', 'TV-MA', 'Youth not allowed', '(Banned)', '17', '19', 'Limited', 'AO', 'X']): lzAR.append('R') # 18
  else: lzAR.append(ar)

movies_sample['age_rating_pp']= lzAR
print(movies_sample['age_rating_pp'].unique())

['All' '15' 'R' 'PG-13' '7']


# 3. Spark Preprocessing
> 0. movieId
1. title (vectorized)
2. era (<-release year, vectorized)
3. genres (one-hot encoding)
4. tag (vectorized)
5. storyline (vectorized)
6. runtime (hr, minmax-scaled)
7. continent (<-country, one-hot encoding)
8. age-rating (one-hot)

### 3.0 Movies DataFrame

In [146]:
schema= [
    StructField("movieId", StringType(), False),
    StructField("title", StringType(), False),
    StructField("genres", StringType(), True),
    StructField("storyline", StringType(), True),
    StructField("country", StringType(), True),
    StructField("release_year", FloatType(), True),
    StructField("runtime", StringType(), True),
    StructField("age_rating", StringType(), True),
    StructField("tag", StringType(), True)
]

In [147]:
spMovies= spark.createDataFrame(movies_sample, schema= StructType(schema))
spMovies.show()

+-------+--------------------+--------------------+--------------------+--------------+------------+------------------+----------+--------------------+
|movieId|               title|              genres|           storyline|       country|release_year|           runtime|age_rating|                 tag|
+-------+--------------------+--------------------+--------------------+--------------+------------+------------------+----------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|A little boy name...| United States|      1995.0| 1 hour 21 minutes|       All|toy comes to life...|
|      2|      Jumanji (1995)|Adventure|Childre...|Jumanji, one of t...| United States|      1995.0| 1 hour 44 minutes|       All|board game trappe...|
|      3|Grumpier Old Men ...|      Comedy|Romance|Things don't seem...| United States|      1995.0| 1 hour 41 minutes|        15|    sequel boat lake|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|This story based ...| United States| 

### 3.1 title

In [148]:
# Fix Title (A, An, The)
spMovies = spMovies.withColumn(
    "title_fix",
    when(col("title").substr(-5, 5) == ", The", "The " + col("title").substr(1, -6))
    .when(col("title").substr(-4, 4) == ", An", "An " + col("title").substr(1, -5))
    .when(col("title").substr(-3, 3) == ", A", "A " + col("title").substr(1, -4))
    .otherwise(col("title"))
)

spMovies = spMovies.drop("title").withColumnRenamed("title_fix", "title")
spMovies.show()

+-------+--------------------+--------------------+--------------+------------+------------------+----------+--------------------+--------------------+
|movieId|              genres|           storyline|       country|release_year|           runtime|age_rating|                 tag|               title|
+-------+--------------------+--------------------+--------------+------------+------------------+----------+--------------------+--------------------+
|      1|Adventure|Animati...|A little boy name...| United States|      1995.0| 1 hour 21 minutes|       All|toy comes to life...|    Toy Story (1995)|
|      2|Adventure|Childre...|Jumanji, one of t...| United States|      1995.0| 1 hour 44 minutes|       All|board game trappe...|      Jumanji (1995)|
|      3|      Comedy|Romance|Things don't seem...| United States|      1995.0| 1 hour 41 minutes|        15|    sequel boat lake|Grumpier Old Men ...|
|      4|Comedy|Drama|Romance|This story based ...| United States|      1995.0| 2 hours 

In [149]:
# Word2Vec
spMovies = spMovies.withColumn('title_arr', F.split(F.col("title")," "))
word2vec = Word2Vec(vectorSize = 100, minCount = 5,
                    inputCol = 'title_arr', outputCol = 'title_vec')
model = word2vec.fit(spMovies)
spMovies = model.transform(spMovies)
spMovies= spMovies.drop('title_arr')
spMovies= spMovies.drop('title').withColumnRenamed('title_vec', 'title')
spMovies.show()

+-------+--------------------+--------------------+--------------+------------+------------------+----------+--------------------+--------------------+
|movieId|              genres|           storyline|       country|release_year|           runtime|age_rating|                 tag|               title|
+-------+--------------------+--------------------+--------------+------------+------------------+----------+--------------------+--------------------+
|      1|Adventure|Animati...|A little boy name...| United States|      1995.0| 1 hour 21 minutes|       All|toy comes to life...|[-0.0147658269076...|
|      2|Adventure|Childre...|Jumanji, one of t...| United States|      1995.0| 1 hour 44 minutes|       All|board game trappe...|[-0.0207926295697...|
|      3|      Comedy|Romance|Things don't seem...| United States|      1995.0| 1 hour 41 minutes|        15|    sequel boat lake|[-0.0010529104038...|
|      4|Comedy|Drama|Romance|This story based ...| United States|      1995.0| 2 hours 

### 3.2 era

In [150]:
spMovies = spMovies.withColumn( 'era',
    F.when((F.col('release_year') < 1930), F.array(F.lit("S")))
    .when((F.col('release_year') >= 1930) & (F.col('release_year') < 1970), F.array(F.lit("G")))
    .when((F.col('release_year') >= 1970) & (F.col('release_year') < 1990), F.array(F.lit("N")))
    .when((F.col('release_year') >= 1990) & (F.col('release_year') < 2000), F.array(F.lit("B")))
    .when(F.col('release_year').isNull(), F.array(F.lit(None)))
    .otherwise(F.array(F.lit("C")))
)

spMovies = spMovies.drop('release_year')
spMovies.show()

+-------+--------------------+--------------------+--------------+------------------+----------+--------------------+--------------------+---+
|movieId|              genres|           storyline|       country|           runtime|age_rating|                 tag|               title|era|
+-------+--------------------+--------------------+--------------+------------------+----------+--------------------+--------------------+---+
|      1|Adventure|Animati...|A little boy name...| United States| 1 hour 21 minutes|       All|toy comes to life...|[-0.0147658269076...|[B]|
|      2|Adventure|Childre...|Jumanji, one of t...| United States| 1 hour 44 minutes|       All|board game trappe...|[-0.0207926295697...|[B]|
|      3|      Comedy|Romance|Things don't seem...| United States| 1 hour 41 minutes|        15|    sequel boat lake|[-0.0010529104038...|[B]|
|      4|Comedy|Drama|Romance|This story based ...| United States| 2 hours 4 minutes|        18|black american fr...|[-0.0032953992486...|[B]|

In [151]:
# One-hot encoding
lzEra= [['S'], ['G'], ['N'], ['B'], ['C']]

def create_ohe(row, lzEra):
    era= row.era
    lz= [0] * len(lzEra)
    for i in range(len(lzEra)):
        if era== lzEra[i]:
            lz[i] = 1
            break
    return lz
create_ohe_udf = F.udf(lambda row: create_ohe(row, lzEra), ArrayType(IntegerType()))
spMovies = spMovies.withColumn('era_ohe', create_ohe_udf(F.struct([col(x) for x in spMovies.columns])))
spMovies = spMovies.drop('era').withColumnRenamed('era_ohe', 'era')
spMovies.show()

+-------+--------------------+--------------------+--------------+------------------+----------+--------------------+--------------------+---------------+
|movieId|              genres|           storyline|       country|           runtime|age_rating|                 tag|               title|            era|
+-------+--------------------+--------------------+--------------+------------------+----------+--------------------+--------------------+---------------+
|      1|Adventure|Animati...|A little boy name...| United States| 1 hour 21 minutes|       All|toy comes to life...|[-0.0147658269076...|[0, 0, 0, 1, 0]|
|      2|Adventure|Childre...|Jumanji, one of t...| United States| 1 hour 44 minutes|       All|board game trappe...|[-0.0207926295697...|[0, 0, 0, 1, 0]|
|      3|      Comedy|Romance|Things don't seem...| United States| 1 hour 41 minutes|        15|    sequel boat lake|[-0.0010529104038...|[0, 0, 0, 1, 0]|
|      4|Comedy|Drama|Romance|This story based ...| United States| 2 h

In [152]:
spMovies.count()

9697

### 3.3 genres

In [153]:
# spMovies = spMovies.withColumn('genres', F.regexp_replace('genres', '\|', ' '))
# spMovies.show()

In [154]:
# One-hot encoding
dfGenres = spMovies.withColumn("genres", explode(split("genres", "\\|")))
lzGenre = dfGenres.select("genres").distinct().rdd.flatMap(lambda x: x).collect()

def create_ohe(row, lz_genre):
    genres = row.genres
    lz = [0] * len(lz_genre)
    for i in range(len(lz_genre)):
        if lz_genre[i] in genres:
            lz[i] = 1
    return lz
create_ohe_udf = F.udf(lambda row: create_ohe(row, lzGenre), ArrayType(IntegerType()))
spMovies = spMovies.withColumn('genres_ohe', create_ohe_udf(F.struct([col(x) for x in spMovies.columns])))

spMovies = spMovies.drop('genres').withColumnRenamed('genres_ohe', 'genres')
spMovies.show()

+-------+--------------------+--------------+------------------+----------+--------------------+--------------------+---------------+--------------------+
|movieId|           storyline|       country|           runtime|age_rating|                 tag|               title|            era|              genres|
+-------+--------------------+--------------+------------------+----------+--------------------+--------------------+---------------+--------------------+
|      1|A little boy name...| United States| 1 hour 21 minutes|       All|toy comes to life...|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|
|      2|Jumanji, one of t...| United States| 1 hour 44 minutes|       All|board game trappe...|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|
|      3|Things don't seem...| United States| 1 hour 41 minutes|        15|    sequel boat lake|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|
|      4|This story based ...| United States| 2 hours 4 minutes|      

In [155]:
spMovies.count()

9697

### 3.4 tag

In [156]:
# Word2Vec
spMovies = spMovies.withColumn('tag_arr', F.split(F.col("tag")," "))
word2vec = Word2Vec(vectorSize = 100, minCount = 5,
                    inputCol = 'tag_arr', outputCol = 'tag_vec')
model = word2vec.fit(spMovies)
spMovies = model.transform(spMovies)

spMovies= spMovies.drop('tag_arr')
spMovies= spMovies.drop('tag').withColumnRenamed('tag_vec', 'tag')
spMovies.show()

+-------+--------------------+--------------+------------------+----------+--------------------+---------------+--------------------+--------------------+
|movieId|           storyline|       country|           runtime|age_rating|               title|            era|              genres|                 tag|
+-------+--------------------+--------------+------------------+----------+--------------------+---------------+--------------------+--------------------+
|      1|A little boy name...| United States| 1 hour 21 minutes|       All|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|
|      2|Jumanji, one of t...| United States| 1 hour 44 minutes|       All|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|
|      3|Things don't seem...| United States| 1 hour 41 minutes|        15|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|
|      4|This story based ...| United States| 2 hours 4 minutes|      

### 3.5 storyline

In [157]:
# Clean, Remove Stopwords
spMovies= spMovies.withColumn('storyline_clean', lower(F.regexp_replace('storyline', '[^a-zA-Z\\s]', '')))
spMovies= spMovies.drop('storyline')

udf_stopwords= ['a', 'the', 'this', 'movie', 'director', 'and', 'so']
def_stopwords= StopWordsRemover.loadDefaultStopWords("english")

spMovies= spMovies.withColumn("storyline_tok", F.split("storyline_clean", "\\s+"))
remover= StopWordsRemover(stopWords= udf_stopwords, inputCol= "storyline_tok", outputCol="storyline_stop")
spMovies= remover.transform(spMovies)

remover2= StopWordsRemover(stopWords= def_stopwords, inputCol= "storyline_stop", outputCol="storyline_stop2")
spMovies = remover2.transform(spMovies)

spMovies= spMovies.drop('storyline_clean')
spMovies= spMovies.drop('storyline_tok')
spMovies= spMovies.drop('storyline_stop').withColumnRenamed('storyline_stop2', 'storyline')
spMovies.show()

+-------+--------------+------------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+
|movieId|       country|           runtime|age_rating|               title|            era|              genres|                 tag|           storyline|
+-------+--------------+------------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+
|      1| United States| 1 hour 21 minutes|       All|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[little, boy, nam...|
|      2| United States| 1 hour 44 minutes|       All|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[jumanji, one, un...|
|      3| United States| 1 hour 41 minutes|        15|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[things, dont, se...|
|      4| United States| 2 hours 4 minutes|        18|[-0.003295399248

In [158]:
# Word2Vec
word2vec = Word2Vec(vectorSize = 100, minCount = 5,
                    inputCol = 'storyline', outputCol = 'storyline_vec')
model = word2vec.fit(spMovies)
spMovies = model.transform(spMovies)

spMovies= spMovies.drop('storyline_arr')
spMovies= spMovies.drop('storyline').withColumnRenamed('storyline_vec', 'storyline')
spMovies.show()

+-------+--------------+------------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+
|movieId|       country|           runtime|age_rating|               title|            era|              genres|                 tag|           storyline|
+-------+--------------+------------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+
|      1| United States| 1 hour 21 minutes|       All|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[0.01950979779940...|
|      2| United States| 1 hour 44 minutes|       All|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[0.01556924401974...|
|      3| United States| 1 hour 41 minutes|        15|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[0.00345107555040...|
|      4| United States| 2 hours 4 minutes|        18|[-0.003295399248

In [159]:
# # Doc2Vec
# word2vec= Word2Vec(vectorSize=100, minCount=0, inputCol="storyline", outputCol="storyline_vec")
# model= word2vec.fit(spMovies)
# spMovies= model.transform(spMovies)

# doc2vec = spMovies.groupBy("movieId").agg(
#     expr("avg(storyline_vec.values)").alias("storyline_docvec")
# )
# doc2vec.show(truncate=False)

### 3.6 runtime

In [160]:
# Hr&Min -> Hr
spMovies = spMovies.withColumn("runtime_split", split(col("runtime"), " "))
spMovies = spMovies.withColumn(
    "runtime_float",
    when(col("runtime_split").getItem(3).isNotNull(),
         col("runtime_split").getItem(0).cast("float") + col("runtime_split").getItem(2).cast("float") / 60)
    .when(col("runtime_split").getItem(0).cast("float") < 5,
          col("runtime_split").getItem(0).cast("float"))
    .otherwise(col("runtime_split").getItem(0).cast("float") / 60)
)

spMovies = spMovies.drop("runtime_split")
spMovies= spMovies.drop('runtime').withColumnRenamed('runtime_float', 'runtime')
spMovies.show()

+-------+--------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+------------------+
|movieId|       country|age_rating|               title|            era|              genres|                 tag|           storyline|           runtime|
+-------+--------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+------------------+
|      1| United States|       All|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[0.01950979779940...|              1.35|
|      2| United States|       All|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[0.01556924401974...|1.7333333333333334|
|      3| United States|        15|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[0.00345107555040...|1.6833333333333333|
|      4| United States|        18|[-0.0032953992486...|[0, 0, 0, 1, 0

In [161]:
spMovies.filter(spMovies.runtime.isNull()).show()

+-------+-------+----------+-----+---+------+---+---------+-------+
|movieId|country|age_rating|title|era|genres|tag|storyline|runtime|
+-------+-------+----------+-----+---+------+---+---------+-------+
+-------+-------+----------+-----+---+------+---+---------+-------+



In [162]:
# Scaling
assembler= VectorAssembler(inputCols= ['runtime'], outputCol= 'runtime_vec')
spMovies_vec= assembler.transform(spMovies)

scaler= StandardScaler(inputCol= 'runtime_vec', outputCol= "runtime_scaled", withMean= True, withStd= True)
spMovies= scaler.fit(spMovies_vec).transform(spMovies_vec)
spMovies= spMovies.drop('runtime_vec')
spMovies= spMovies.drop('runtime').withColumnRenamed('runtime_scaled', 'runtime')
spMovies.show()

+-------+--------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+
|movieId|       country|age_rating|               title|            era|              genres|                 tag|           storyline|             runtime|
+-------+--------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+
|      1| United States|       All|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[0.01950979779940...|[-0.8024960503192...|
|      2| United States|       All|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[0.01556924401974...|[-0.0494984879328...|
|      3| United States|        15|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[0.00345107555040...|[-0.1477155612875...|
|      4| United States|        18|[-0.0032953992486...|[0

### 3.7 continent

Asia, Africa, North America, South America, Europe, Oceania



* Russia - asia, europe
* Sweden - europe
* Germany - europe
* France - europe
* Yugoslavia - europe
* Luxembourg - europe
* Belgium - europe
* Italy - europe
* Norway - europe
* Spain - europe
* Denmark - europe
* Ireland - europe
* Monaco - europe
* Hungary - europe
* Czechoslovakia - europe
* West Germany  - europe
* United Kingdom - europe
* Netherlands - europe
* Poland - europe
* Switzerland - europe
* Czech Republic  - europe

* Japan - asia
* Russia - asia, europe
* Taiwan - asia
* India - asia
* Bhutan - asia
* China - asia
* Iran - asia
* Hong Kong - asia

* United States - north_america
* Canada - north_america
* Aruba - north_america

* Mexico - south_america
* Brazil- south_america

* New Zealand - oceania
* Australia - oceania

* South Africa - africa






In [163]:
lzEU= ['Russia', 'Sweden', 'Germany',
       'France', 'Yugoslavia', 'Luxembourg',
       'Belgium', 'Italy', 'Norway',
       'Spain', 'Denmark', 'Ireland',
       'Monaco', 'Hungary', 'Czechoslovakia',
       'West Germany', 'United Kingdom', 'Netherlands',
       'Poland', 'Switzerland', 'Czech Republic']
lzAS= ['Japan', 'Taiwan', 'India', 'Bhutan',
       'China', 'Iran', 'Hong Kong',]
lzNA= ['United States', 'Canada', 'Aruba']
lzSA= ['Mexico', 'Brazil']
lzOC= ['New Zealand', 'Australia']
lzAF= ['South Africa']

spMovies = spMovies.withColumn(
    'continent',
    when(spMovies['country'].isin(lzEU), 'EU')
    .when(spMovies['country'].isin(lzAS), 'AS')
    .when(spMovies['country'].isin(lzNA), 'NA')
    .when(spMovies['country'].isin(lzSA), 'SA')
    .when(spMovies['country'].isin(lzOC), 'OC')
    .otherwise('AF')
)
spMovies= spMovies.drop('country')
spMovies.show()

+-------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------+
|movieId|age_rating|               title|            era|              genres|                 tag|           storyline|             runtime|continent|
+-------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------+
|      1|       All|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[0.01950979779940...|[-0.8024960503192...|       NA|
|      2|       All|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[0.01556924401974...|[-0.0494984879328...|       NA|
|      3|        15|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[0.00345107555040...|[-0.1477155612875...|       NA|
|      4|        18|[-0.0032953992486...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 1, 0...|[-0.008344

In [164]:
# One-hot encoding
lzCont= ['AS', 'AF', 'EU', 'NA', 'OC', 'SA']

def create_ohe(row, lzCont):
    cont= row.continent
    lz= [0] * len(lzCont)
    for i in range(len(lzCont)):
        if cont== lzCont[i]:
            lz[i] = 1
            break
    return lz
create_ohe_udf = F.udf(lambda row: create_ohe(row, lzCont), ArrayType(IntegerType()))
spMovies = spMovies.withColumn('continent_ohe', create_ohe_udf(F.struct([col(x) for x in spMovies.columns])))

spMovies = spMovies.drop('continent').withColumnRenamed('continent_ohe', 'continent')
spMovies.show()

+-------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+------------------+
|movieId|age_rating|               title|            era|              genres|                 tag|           storyline|             runtime|         continent|
+-------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+------------------+
|      1|       All|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[0.01950979779940...|[-0.8024960503192...|[0, 0, 0, 1, 0, 0]|
|      2|       All|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[0.01556924401974...|[-0.0494984879328...|[0, 0, 0, 1, 0, 0]|
|      3|        15|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[0.00345107555040...|[-0.1477155612875...|[0, 0, 0, 1, 0, 0]|
|      4|        18|[-0.0032953992

### 3.8 age_rating

In [165]:
# Ohe-hot encoding
lzAR= ['All', '7', 'PG-13', '15', 'R']

def create_ohe(row, lzAR):
    ar= row.age_rating
    lz= [0] * len(lzAR)
    for i in range(len(lzAR)):
        if ar== lzAR[i]:
            lz[i] = 1
            break
    return lz
create_ohe_udf = F.udf(lambda row: create_ohe(row, lzAR), ArrayType(IntegerType()))
spMovies = spMovies.withColumn('age_rating_ohe', create_ohe_udf(F.struct([col(x) for x in spMovies.columns])))

spMovies = spMovies.drop('age_rating').withColumnRenamed('age_rating_ohe', 'age_rating')
spMovies.show()

+-------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+------------------+---------------+
|movieId|               title|            era|              genres|                 tag|           storyline|             runtime|         continent|     age_rating|
+-------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+------------------+---------------+
|      1|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[0.01950979779940...|[-0.8024960503192...|[0, 0, 0, 1, 0, 0]|[1, 0, 0, 0, 0]|
|      2|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[0.01556924401974...|[-0.0494984879328...|[0, 0, 0, 1, 0, 0]|[1, 0, 0, 0, 0]|
|      3|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[0.00345107555040...|[-0.1477155612875...|[0, 0, 0, 1, 0, 0]|[0, 0, 0, 1, 0]|
|   

# 4. Export

In [166]:
spMovies.show()

+-------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+------------------+---------------+
|movieId|               title|            era|              genres|                 tag|           storyline|             runtime|         continent|     age_rating|
+-------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+------------------+---------------+
|      1|[-0.0147658269076...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0641816549818...|[0.01950979779940...|[-0.8024960503192...|[0, 0, 0, 1, 0, 0]|[1, 0, 0, 0, 0]|
|      2|[-0.0207926295697...|[0, 0, 0, 1, 0]|[0, 0, 0, 1, 0, 0...|[-0.0252036253223...|[0.01556924401974...|[-0.0494984879328...|[0, 0, 0, 1, 0, 0]|[1, 0, 0, 0, 0]|
|      3|[-0.0010529104038...|[0, 0, 0, 1, 0]|[0, 1, 0, 0, 0, 0...|[-0.0236170838276...|[0.00345107555040...|[-0.1477155612875...|[0, 0, 0, 1, 0, 0]|[0, 0, 0, 1, 0]|
|   

In [170]:
dfMovies= spMovies.toPandas()
dfMovies.head()

,movieId,title,era,genres,tag,storyline,runtime,continent,age_rating
0,1,"[-0.014765826907629767, -0.0001763248195250829...","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...","[-0.06418165498180316, -0.009971046587452292, ...","[0.019509797799400985, -0.022897314243508795, ...",[-0.8024960503192684],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"
1,2,"[-0.020792629569768906, -0.005490744020789862,...","[0, 0, 0, 1, 0]","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[-0.025203625322319567, -0.023397489140431084,...","[0.015569244019747148, -0.0014084530279929327,...",[-0.04949848793283062],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"
2,3,"[-0.0010529104038141668, -0.003023075812961906...","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.02361708382765452, -0.006075011993137498, ...","[0.0034510755504015833, -0.013722939333092654,...",[-0.14771556128758348],"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 1, 0]"
3,4,"[-0.003295399248600006, -0.007947875885292888,...","[0, 0, 0, 1, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.008344948175363243, -0.024232767755165696,...","[0.005609218422950317, -0.022722859384838298, ...",[0.6052820010988548],"[0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0]"
4,5,"[0.007662885689309664, -0.004842474711559979, ...","[0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.013301791623234749, -0.008611441900332768,...","[0.004049535797726687, -0.010654402755038893, ...",[0.015979560970337654],"[0, 0, 0, 1, 0, 0]","[1, 0, 0, 0, 0]"


In [171]:
dfMovies.to_csv(path+'1210 all_pp.csv', encoding= 'utf-8', index= False)